## Extraction Bench: Lesson 1
### Using a non-polar solute to extract a solute from water

In this tutorial, I am going to walk you through how our extraction environment works and hopefully give some insight into how an
RL agent might interact with the environment. In this extraction we are going to be using water to extract sodium and
chlorine from oil. We are going to be using this jupyter notebook in order to interact
with the environment.

In [ ]:
!pip install "git+https://github.com/chemgymrl/chemgymrl.git@manager"

In [ ]:
import sys
sys.path.append('../../')
from chemistrylab import material, vessel
from chemistrylab.benches.general_bench import *
from chemistrylab.util.reward import RewardGenerator
from chemistrylab.lab.shelf import Shelf
from chemistrylab.util import Visualization
from matplotlib import pyplot as plt
Visualization.use_mpl_light(size=2)

# Creating a Shelf filled with Vessels

Here we create a shelf containing all of the vessels required for this material. The extraction vessel will have NaCl dissolved in oil, two extra vessels will be provided, one to extract the salt and the other to pour out any waste material (oil).

In [ ]:
def make_solvent(mat):
    "Makes a Vessel with 2 Litres of a single material"

    solvent = material.REGISTRY[mat](mol=1.0)

    solvent_vessel = vessel.Vessel(
        label=f'{solvent._name} Vessel',
    )
    #Setting the mol st there are 2 litres
    solvent.mol=(2/solvent.volume_L)
    solvent_vessel.material_dict = {mat:solvent}
    # instruct the vessel to update its material dictionary
    solvent_vessel.volume = solvent_vessel.filled_volume()
    return solvent_vessel


extraction_vessel = vessel.Vessel(label='Extract Vessel')
# initialize H2O
C6H14 = material.REGISTRY["CCCCCC"](mol=1)
# Get dissolved NaCl
Na = material.REGISTRY["[Na+]"](mol=1)
Cl = material.REGISTRY["[Cl-]"](mol=1)

mats = [C6H14,Na,Cl]
# Set up the vessel
extraction_vessel.material_dict={str(mat):mat for mat in mats}
extraction_vessel.validate_solvents()
extraction_vessel.validate_solutes()

shelf = Shelf([  
    extraction_vessel,
    vessel.Vessel("Extract Beaker"),
    vessel.Vessel("Waste"),
    make_solvent("CCCCCC"),
    make_solvent("O")
], n_working = 2)

print(shelf)

# Creating the Actions

Actions are parameterized by a named tuple:
```python
class Action(NamedTuple):
    vessels: Tuple[int]
    parameters: Tuple[tuple]
    event_name: str
    affected_vessels: Optional[Tuple[int]]
    dt: float
    terminal: bool
```



In [ ]:

#Recall the shelf:         0                1         2          3           4
#           Shelf: (Extract Vessel, Extract Beaker, Waste, C6H14 Vessel, H2O Vessel)

#Setting the volumes to pour (in liters)
amounts=np.linspace(0.2,1,5).reshape([5,1])
#setting the pixels to drain (each pixel represents 10ml of draining in this case)
pixels = np.arange(2,12,2).reshape([5,1])
        
actions = [
    # Pouring the extraction vessel into the extraciton beaker
    Action([0], pixels,              'drain by pixel',[1],  0.01, False),
    # Mixing the extraction vessel
    Action([0],-amounts,             'mix',           None, 0.00, False),
    # Pouring the extraction beaker into the extraction vessel
    Action([1], amounts,             'pour by volume',[0],  0.01, False),
    # Pouring the waste vessel into the extraction vessel
    Action([2], amounts,             'pour by volume',[0],  0.01, False),
    # Pouring the extraction vessel into the waste beaker
    Action([0], amounts,             'pour by volume',[2],  0.01, False),
    # Pouring C6H14 into the extraction vessel
    Action([3], amounts/2,           'pour by volume',[0],  0,    False),
    # Pouring Water into the extraction vessel
    Action([4], amounts/2,           'pour by volume',[0],  0,    False),
    # Waiting for vessels to settle
    Action([0,1,2], 32**amounts/200, 'mix',           None, 0,    False),
    # Ending the experiment
    Action([0], [[0]],               'mix',           None, 0,    True)
]



# Determining the Reward Scheme & observations

Since we just want to get the salt out of the oil, we have our reward function discount the reward if C6H14 is present, and set NaCl as our target, making sure to include dissolved components as NaCl. In order to see what we are doing with the extraction, we will set the observation to just return layer information of our vessels.

In [ ]:
e_rew= RewardGenerator(use_purity=False, exclude_solvents=True, include_dissolved=True, exclude_mat="CCCCCC")
targets = ["[Na+].[Cl-]"]

#Just show layer info
observations = ["layers"]


# Making the Bench

In [ ]:
water_oil_bench = GenBench(
    shelf,
    actions,
    observations,
    targets=targets,
    reward_function=e_rew,
)

# Running the Bench

In [ ]:
_ = water_oil_bench.reset()

plt.imshow(water_oil_bench.render())
plt.axis("off")
plt.show()


In [ ]:
#Pour in water
obs,rew,d,*_ = water_oil_bench.step(33)
obs,rew,d,*_ = water_oil_bench.step(39)
obs,rew,d,*_ = water_oil_bench.step(39)
plt.imshow(water_oil_bench.render())
plt.axis("off")
plt.show()

Now that we've added the water we need to mix the vessel to get the solutes to transfer into the oil, so let's mix the
vessel! As seen in the graph below we can see that based on the layer representation that we have mixed the oil and the water.


In [ ]:
# Mix
obs,rew,d,*_ = water_oil_bench.step(9)

plt.imshow(water_oil_bench.render())
plt.axis("off")
plt.show()

Now that we have done some mixing we need to wait for the oil to settle to the top of the water so we can drain the
water. Keep repeating the following command until the graph settles.


In [ ]:
obs,rew,d,*_ = water_oil_bench.step(39)
obs,rew,d,*_ = water_oil_bench.step(39)
plt.imshow(water_oil_bench.render())
plt.axis("off")
plt.show()

Now that the water and oil have settled we want to drain out our water into beaker 1 so that we can pour out our oil
out as waste.



In [ ]:
#Pouring
for i in range(4):
    obs,rew,d,*_=water_oil_bench.step(4)

obs,rew,d,*_=water_oil_bench.step(1)
    
plt.imshow(water_oil_bench.render())
plt.axis("off")
plt.show()

Now we just have to empty out the extraction vessel into the waste vessel to get rid of the oil and we are done

In [ ]:
obs,rew,d,*_= water_oil_bench.step(24)

obs,final_return,d,*_ = water_oil_bench.step(40)

print(final_return,d)

Now if we want to we can pour back the water from vessel 1 into our extraction vessel and repeat the process to get a
more of the sodium out of the oil. However, for an introduction this much should satisfy, now that we have finished,
we want to see how well we did so now we enter the done command.

I hope this tutorial helped with your understanding of how an agent might interact with the extraction environmenment!